In [1]:
# imports

import pandas as pd
import sentencepiece as spm
import tempfile

In [2]:
# load data

column_names = ['ref', '1', '2', '3', '4', 'text']
jorf_2023 = pd.read_csv('jorf_2023.csv', sep='|', names=column_names, index_col=False)

In [3]:
# format input

fraction = 0.1 # fraction of the corpus to use for training
save_input = False # True if you want to save the input file for sentencepiece

text = jorf_2023['text'].tolist()[:int(len(jorf_2023)*fraction)]

if save_input:
    with open('input.txt', 'w') as f:
        for line in text:
            f.write(line + '\n')

In [4]:
# train model

model_prefix = 'jorf'

if save_input:
    temp_path = 'input.txt'

else:
    with tempfile.NamedTemporaryFile(delete=False, mode='w') as temp:
        for line in text:
            temp.write(line + '\n')
        temp_path = temp.name

spm.SentencePieceTrainer.train(f'--input={temp_path} --model_prefix={model_prefix} --vocab_size=1000')

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=/var/folders/9h/pf8tzcpd2w56l_xpcr9j2z_40000gn/T/tmpt_29gizx --model_prefix=jorf --vocab_size=1000
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /var/folders/9h/pf8tzcpd2w56l_xpcr9j2z_40000gn/T/tmpt_29gizx
  input_format: 
  model_prefix: jorf
  model_type: UNIGRAM
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  

In [5]:
# load model

sp = spm.SentencePieceProcessor()
sp.load(f'{model_prefix}.model')

True

In [6]:
# test encode

sub_units = sp.encode_as_pieces("Art. R. 228-1. — Les juristes assistants recrutés en application de l'article L. 228-1 apportent leur concours à l'analyse juridique des dossiers nécessitant une expertise particulière qui leur sont confiés par les magistrats sous la direction desquels ils sont placés. Ils sont recrutés en qualité d'agent contractuel de l'Etat relevant de la catégorie A.")
ids = sp.encode_as_ids("Art. R. 228-1. — Les juristes assistants recrutés en application de l'article L. 228-1 apportent leur concours à l'analyse juridique des dossiers nécessitant une expertise particulière qui leur sont confiés par les magistrats sous la direction desquels ils sont placés. Ils sont recrutés en qualité d'agent contractuel de l'Etat relevant de la catégorie A.")
print(sub_units)
print(ids)

['▁Art', '.', '▁R', '.', '▁', '22', '8', '-1', '.', '▁', '—', '▁Les', '▁', 'j', 'ur', 'is', 'te', 's', '▁a', 's', 's', 'is', 't', 'ant', 's', '▁re', 'c', 'r', 'ut', 'és', '▁en', '▁application', '▁de', '▁l', "'", 'article', '▁L', '.', '▁', '22', '8', '-1', '▁', 'ap', 'p', 'ort', 'ent', '▁leur', '▁concours', '▁à', '▁l', "'", 'analyse', '▁', 'j', 'ur', 'i', 'd', 'ique', '▁des', '▁dossier', 's', '▁n', 'éc', 'es', 's', 'it', 'ant', '▁une', '▁', 'expert', 'is', 'e', '▁par', 't', 'ic', 'ul', 'ière', '▁qui', '▁leur', '▁sont', '▁con', 'f', 'i', 'és', '▁par', '▁les', '▁m', 'ag', 'is', 't', 'r', 'at', 's', '▁sous', '▁la', '▁direction', '▁des', 'quel', 's', '▁', 'il', 's', '▁sont', '▁p', 'l', 'ac', 'és', '.', '▁Il', 's', '▁sont', '▁re', 'c', 'r', 'ut', 'és', '▁en', '▁qualité', '▁d', "'", 'ag', 'ent', '▁contractuel', '▁de', '▁l', "'", 'Etat', '▁relevant', '▁de', '▁la', '▁catégorie', '▁A', '.']
[274, 9, 107, 9, 4, 303, 89, 150, 9, 4, 46, 124, 4, 239, 125, 39, 106, 3, 37, 3, 3, 39, 13, 49, 3, 77, 31,

In [7]:
# test decode 
print(sp.decode_pieces(sub_units))
print(sp.decode_ids(ids))

Art. R. 228-1. — Les juristes assistants recrutés en application de l'article L. 228-1 apportent leur concours à l'analyse juridique des dossiers nécessitant une expertise particulière qui leur sont confiés par les magistrats sous la direction desquels ils sont placés. Ils sont recrutés en qualité d'agent contractuel de l'Etat relevant de la catégorie A.
Art. R. 228-1. — Les juristes assistants recrutés en application de l'article L. 228-1 apportent leur concours à l'analyse juridique des dossiers nécessitant une expertise particulière qui leur sont confiés par les magistrats sous la direction desquels ils sont placés. Ils sont recrutés en qualité d'agent contractuel de l'Etat relevant de la catégorie A.
